In [1]:
from __future__ import (unicode_literals, print_function, absolute_import,
                        division)


from ema_workbench import (Model, MultiprocessingEvaluator, Policy,
                           Scenario, IntegerParameter)
from ema_workbench.em_framework.evaluators import BaseEvaluator

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.optimization import (HyperVolume,
                                                     EpsilonProgress)
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time
from problem_formulation import get_model_for_problem_formulation
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


In [2]:

results = pd.read_csv("Optimizationresults 100K.csv") 
#Sum the different RfR values to identify the one with 0 for the our municiaplites. identify intresting solutions

results ["sum rfr"] = (results ['0_RfR 0'] + 
                       results ['0_RfR 1'] +
                       results ['0_RfR 2'] +
                       results ['1_RfR 0'] + 
                       results ['1_RfR 1'] +
                       results ['1_RfR 2'] +
                       results ['2_RfR 0'] + 
                       results ['2_RfR 1'] +
                       results ['2_RfR 2'])


# This code selects preferences based on the municipality preferences
#no RFR costs for dikes
#no deaths in dike 1
#0.00148 for dike 2
#0.001 for dike 3 
int_solutions = (
                (results["sum rfr"] <= 0) & 
                (results['A.1_Expected Number of Deaths'] <= 0.0) &
                (results['A.2_Expected Number of Deaths'] <= 0.00148) &
                (results['A.3_Expected Number of Deaths'] <= 0.001)
                )
int_solutions.value_counts()
results['Satisfying'] = int_solutions
satisfying_solutions = results[results['Satisfying']==True]

int_solutions = satisfying_solutions
int_solutions

,Unnamed: 0,0_RfR 0,0_RfR 1,0_RfR 2,1_RfR 0,1_RfR 1,1_RfR 2,2_RfR 0,2_RfR 1,2_RfR 2,...,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs,sum rfr,Satisfying
277,277,0,0,0,0,0,0,0,0,0,...,1.261015e+08,0.0,2.225849e+07,0.000000,3.337712e+07,0.000000,0.0,0.000000,0,True
558,558,0,0,0,0,0,0,0,0,0,...,3.422156e+07,0.0,6.348950e+06,0.004159,2.503721e+07,0.000000,242400000.0,0.000000,0,True
683,683,0,0,0,0,0,0,0,0,0,...,6.226744e+07,0.0,1.428414e+07,0.003766,1.087498e+08,0.065523,0.0,0.000000,0,True
832,832,0,0,0,0,0,0,0,0,0,...,2.879840e+07,0.0,1.808200e+07,0.000000,1.134436e+08,0.014069,0.0,5103.412196,0,True
1221,1221,0,0,0,0,0,0,0,0,0,...,1.362141e+08,0.0,1.692524e+07,0.000000,1.377821e+08,0.070516,0.0,0.000000,0,True


In [3]:
opt_results.keys()

NameError: name 'opt_results' is not defined

In [ ]:
from ema_workbench.analysis import parcoords


data = int_solutions.loc[:, ['A.1 Total Costs', 'A.2 Total Costs',
               'A.3 Total Costs','A.4 Total Costs',
               'A.5 Total Costs','RfR Total Costs',
               'Expected Evacuation Costs']]
limits = parcoords.get_limits(data)
limits.loc[0, ['A.1 Total Costs', 'A.2 Total Costs',
               'A.3 Total Costs','A.4 Total Costs',
               'A.5 Total Costs','RfR Total Costs',
               'Expected Evacuation Costs']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
# paraxes.invert_axis('max_P')
plt.show()

In [ ]:
from ema_workbench.analysis import parcoords

data = int_solutions.loc[:, ['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths',
                           'A.3_Expected Number of Deaths','A.4_Expected Number of Deaths',
                           'A.5_Expected Number of Deaths']]
limits = parcoords.get_limits(data)
limits.loc[0, ['A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths',
                           'A.3_Expected Number of Deaths','A.4_Expected Number of Deaths',
                           'A.5_Expected Number of Deaths']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
# paraxes.invert_axis('max_P')
plt.show()

In [ ]:
int_solutions.to_csv(r'intsolutions.csv')

In [ ]:
from ema_workbench import load_results

results = load_results('SECOND candidate solutions MORDM 1000scenarios.tar.gz')
experiments, outcomes = results

In [ ]:
experiments.head()


In [ ]:
convergence = pd.read_csv("Convergenceresults 100K.csv") 

fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True, figsize=(8,4))
ax1.plot(convergence.nfe, convergence.epsilon_progress)
ax1.set_ylabel('$\epsilon$-progress')
ax1.set_xlabel('number of function evaluations') 
plt.show()